In [1]:
import re
from collections import defaultdict
from docx import Document
import pandas as pd

# Define the category mapping
category_map = {
    "Танки": "tanks",
    "ББМ": "bbm",
    "Артилерійські системи": "artillery_systems",
    "РСЗВ": "rszv",
    "Засоби ППО": "air_defense",
    "Літаки": "aircraft",
    "Гелікоптери": "helicopters",
    "БПЛА": "uavs",
    "Ракети": "cruise_missiles",
    "Кораблі (катери)": "ships",
    "Підводні човни": "submarines",
    "Автомобілі та автоцистерни": "vehicles",
    "Спеціальна техніка": "special_equipment",
    "Особовий склад": "personnel"
}

# Load the data from the file
file_path = '../input/data.docx'

def extract_text_from_docx(file_path):
    # Initialize the Document object to read the file
    doc = Document(file_path)
    full_text = []
    # Iterate through each paragraph in the document
    for para in doc.paragraphs:
        # Append the text of each paragraph to the full_text list
        full_text.append(para.text)
    # Join the paragraphs with newline characters and return the full text
    return '\n'.join(full_text)

# Extract text from the DOCX file
data = extract_text_from_docx(file_path)

# Initialize a defaultdict to store the parsed data
parsed_data = defaultdict(dict)

# Define regular expression patterns to match date lines, category lines, and personnel lines
date_pattern = re.compile(r'(\d{2}\.\d{2}\.\d{4})')
category_pattern = re.compile(r'([А-Яа-яІіЄєЇїҐґA-Za-z\s]+)\s+—\s+(\d+)(?:\s+\(\+\d+\))?')
personnel_pattern = re.compile(r'Особовий склад\s+—\s+(?:близько\s+|більше\s+)?(\d+)(?:\s+осіб)?(?:\s+\(\+\д+\))?')

# Split the data into sections by dates using the date_pattern
sections = re.split(date_pattern, data.strip())

# Process each section
for i in range(1, len(sections), 2):
    # Extract the date
    date = sections[i]
    # Split the corresponding categories and values by newline
    entries = sections[i + 1].strip().split('\n')
    # Process each entry in the section
    for entry in entries:
        entry = entry.strip()
        # Match the entry against the category pattern or personnel pattern
        if 'Особовий склад' in entry:
            match = personnel_pattern.match(entry)
        else:
            match = category_pattern.match(entry)
        if match:
            # Extract the category and value from the match groups
            groups = match.groups()
            if len(groups) == 2:
                category, value = groups
            elif len(groups) == 1:
                category = "Особовий склад"
                value = groups[0]
            # Map the category to its English equivalent using category_map
            english_category = category_map.get(category.strip(), category.strip())
            # Store the value in parsed_data under the corresponding date and category
            parsed_data[date][english_category] = int(value)

# Convert defaultdict to a regular dictionary
parsed_data = dict(parsed_data)

# Get the list of all unique categories in English
all_categories = set(category_map.values())

# Create a dataset using pandas DataFrame with columns 'date' and sorted categories
columns = ['date'] + sorted([cat for cat in all_categories if cat != "personnel"]) + ["personnel"]
df = pd.DataFrame(columns=columns)

# Populate the DataFrame with parsed data
rows = []
seen_dates = set()
for date, categories in parsed_data.items():
    if date not in seen_dates:
        seen_dates.add(date)
        # Create a row dictionary with the date
        row = {'date': date}
        # Update the row with the categories and their values
        row.update(categories)
        # Append the row to the rows list
        rows.append(row)

# Convert the rows list to a DataFrame
df = pd.DataFrame(rows, columns=columns)

# Fill NaN values with 0 and ensure all values except the date are integers
for column in df.columns:
    if column != 'date':
        df[column] = df[column].fillna(0).astype(int)

# Print the dataset
print("Original Dataset:")
print(df)

# Save the original dataset to a CSV file
df.to_csv('../data/parsed_data.csv', index=False)

# Sort the DataFrame by date to ensure the correct order for calculating differences
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df = df.sort_values(by='date')

# Calculate the difference between each row and the previous row, keeping the first row as is
df_diff = df.copy()
df_diff.iloc[1:, 1:] = df.iloc[1:, 1:].diff().fillna(0).astype(int)

# Sort the difference dataset by date in descending order
df_diff = df_diff.sort_values(by='date', ascending=False)

# Convert the 'date' column back to string format
df_diff['date'] = df_diff['date'].dt.strftime('%d.%m.%Y')

# Print the difference dataset
print("Difference Dataset (sorted by date DESC):")
print(df_diff)

# Save the difference dataset to a CSV file
df_diff.to_csv('../data/df_daily_casualties_russia.csv', index=False)

# Save the difference dataset to a JSON file
df_diff.to_json('../data/df_daily_casualties_russia.json', orient='records', lines=True)

Original Dataset:
           date  air_defense  aircraft  artillery_systems    bbm  \
0    03.06.2024          824       357              13280  15002   
1    02.06.2024          821       357              13233  14980   
2    01.06.2024          815       357              13184  14947   
3    31.05.2024          815       357              13140  14935   
4    30.05.2024          815       357              13101  14913   
..          ...          ...       ...                ...    ...   
826  28.02.2022            0        29                 74    816   
827  27.02.2022            0        27                  0    706   
828  26.02.2022            0        16                  0      0   
829  25.02.2022            0        10                  0    516   
830  24.02.2022            0         7                  0    130   

     cruise_missiles  helicopters  rszv  ships  special_equipment  submarines  \
0               2268          326  1090      0               2199           1   
1  